<a href="https://colab.research.google.com/github/mehrseno/Paraphrase-Identification/blob/main/Paraphrase_Identification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install --upgrade gensim
!pip install numpy==1.20.3

In [ ]:
import pandas as pd

# train https://drive.google.com/file/d/17eLq5Ng5yfbX9tLq2tdiOdscO0ea8xK5/view?usp=sharing
# valid https://drive.google.com/file/d/1Y-68UagW14hwJXNyaR2HnML5jEwvjQlj/view?usp=sharing
# test https://drive.google.com/file/d/1P_HLLvGq15gsgDl5P6QYrHkTlkcytMdr/view?usp=sharing

!gdown --id 17eLq5Ng5yfbX9tLq2tdiOdscO0ea8xK5
!gdown --id 1Y-68UagW14hwJXNyaR2HnML5jEwvjQlj
!gdown --id 1P_HLLvGq15gsgDl5P6QYrHkTlkcytMdr

train = pd.read_csv("train.csv")
valid = pd.read_csv("valid.csv")
test = pd.read_csv("test.csv")

In [ ]:
# remove punctuation marks, non-alphabetic tokens, and lowercase the tokens
def clean_data(data):

  tokenized_data = []

  for d in data:
    tokens = d.split()
    translation_table = str.maketrans('', '', "\"#$%&'()*+,-/:;<=>?@[\]^_`{|}~،؟!❊#$٪^&*)(ـ+=-؛:{}")
    tokens = [w.translate(translation_table) for w in tokens]
    tokens = [word for word in tokens if word.isalpha()]
    tokenized_data.append(tokens)

  return tokenized_data

In [ ]:
import numpy as np

length = []

for index, row in train.iterrows():
  length.append(len(row.sentence1.split(" ")))
  length.append(len(row.sentence2.split(" ")))

print(np.mean(length))

18.972063232488416


In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

MAX_SEQ_LENGTH = 20

# clean data
cleaned_train_data = clean_data(train.sentence1.append(train.sentence2))
cleaned_valid_data = clean_data(valid.sentence1.append(valid.sentence2))
cleaned_test_data = clean_data(test.sentence1.append(test.sentence2))

# create the tokenizer and fit it on the input text
tokenizer = Tokenizer()
tokenizer.fit_on_texts(cleaned_train_data)

# tokenize the input text into encoded numbers
train_sequences = tokenizer.texts_to_sequences(cleaned_train_data)
valid_sequences = tokenizer.texts_to_sequences(cleaned_valid_data)
test_sequences = tokenizer.texts_to_sequences(cleaned_test_data)

# pad the input text 
train_sequences = pad_sequences(train_sequences, maxlen=MAX_SEQ_LENGTH, padding='post')
valid_sequences = pad_sequences(valid_sequences, maxlen=MAX_SEQ_LENGTH, padding='post')
test_sequences = pad_sequences(test_sequences, maxlen=MAX_SEQ_LENGTH, padding='post')

# separate sentence1 and sentence2
train_sent1 = train_sequences[:len(train)]
train_sent2 = train_sequences[len(train): len(train) * 2]
valid_sent1 = valid_sequences[:len(valid)]
valid_sent2 = valid_sequences[len(valid): len(valid) * 2]
test_sent1 = test_sequences[:len(test)]
test_sent2 = test_sequences[len(test): len(test) * 2]

In [ ]:
word_index = tokenizer.word_index
VOCAB_SIZE = len(word_index) + 1
EMB_SIZE = 100

In [ ]:
# train word2vec 
from gensim.models import Word2Vec

train_words = clean_data(train.sentence1.append(train.sentence2))

w2v_model = Word2Vec(train_words, min_count=1, vector_size=EMB_SIZE, window=3, sg=1)

words = list(w2v_model.wv.key_to_index.keys())
embeddings = w2v_model.wv.vectors

lines = []
for i in range(len(words)):
    line = words[i]
    for j in range(len(embeddings[i])):
        line += ' '
        line += str(embeddings[i][j])
    lines.append(line)

with open('drive/MyDrive/MCI/HW5/MRPC.' + str(EMB_SIZE) + '.vec', 'w') as f:
    for l in lines:
        f.write("%s\n" % l)

In [ ]:
import numpy as np

embeddings_index = {}

f = open('drive/MyDrive/MCI/HW5/MRPC.' + str(EMB_SIZE) + '.vec')

for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

embedding_matrix = np.random.random((VOCAB_SIZE, EMB_SIZE))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import *
from tensorflow.keras import optimizers, losses
from tensorflow.keras import backend as K
from tensorflow.keras import initializers, regularizers, constraints
from tensorflow.keras.metrics import *
# from keras_self_attention import SeqSelfAttention

In [ ]:
class Attention(Layer):
    def __init__(self, **kwargs):
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):

        self.W = self.add_weight(name="attention_weight", shape=(input_shape[-1], 1), initializer="normal")
        self.b = self.add_weight(name="attention_bias", shape=(input_shape[1], 1), initializer="zeros")      

        super(Attention, self).build(input_shape)

    def call(self, x):

        et = K.squeeze(K.tanh(K.dot(x, self.W) + self.b), axis=-1)
        at = K.softmax(et)
        at = K.expand_dims(at, axis=-1)
        output = x * at
        # return K.sum(output, axis=1)
        return output

    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[-1])

In [ ]:
# define model
def get_model(VOCAB_SIZE, EMB_SIZE, MAX_SEQ_LENGTH, embedding_matrix):

  # define layers
  embedding_layer = Embedding(VOCAB_SIZE, EMB_SIZE, input_length=MAX_SEQ_LENGTH, weights=[embedding_matrix], mask_zero=True, name='EmbeddingLayer')
  bilstm_layer = Bidirectional(LSTM(EMB_SIZE, dropout=0.2, activation="tanh", return_sequences=True, recurrent_dropout=0.5))
  att_layer = Attention()


  # sentence1
  sent1_input = Input(shape=(MAX_SEQ_LENGTH,), name='Sentence1')
  sent1_emb = embedding_layer(sent1_input)
  sent1_bilstm = bilstm_layer(sent1_emb)
  sent1_att = att_layer(sent1_bilstm)


  # sentence2
  sent2_input = Input(shape=(MAX_SEQ_LENGTH,), name='Sentence2')
  sent2_emb = embedding_layer(sent2_input)
  sent2_bilstm = bilstm_layer(sent2_emb)
  sent2_att = att_layer(sent2_bilstm)
  
 
  concat_layer = Concatenate(name='concatenate', axis=1)([sent1_att, sent2_att])
  flatten = Flatten()(concat_layer)

  drop_layer1 = Dropout(0.2)(flatten)
  dense_layer1 = Dense(512, activation='relu', name='DenseLayer1')(drop_layer1)
  dense_layer2 = Dense(128, activation='relu', name='DenseLayer2')(drop_layer1)
  dense_layer3 = Dense(64, activation='relu', name='DenseLayer3')(dense_layer2)
  dense_layer4 = Dense(32, activation='relu', name='DenseLayer4')(dense_layer3)
  drop_layer2 = Dropout(0.2)(dense_layer4)

  out = Dense(2, activation='softmax', name='OutputLayer')(drop_layer2)

  model = Model(inputs=[sent1_input, sent2_input], outputs=out)

  optimizer = optimizers.Adam(learning_rate=2e-4)
  loss = losses.binary_crossentropy

  model.compile(loss=loss, optimizer=optimizer, metrics=['accuracy', Recall(), Precision()])

  return model

In [ ]:
model = get_model(VOCAB_SIZE, EMB_SIZE, MAX_SEQ_LENGTH, embedding_matrix)
model.summary()

In [ ]:
from tensorflow.keras.utils import to_categorical

y_train = np.array(to_categorical(train.label), dtype=int)
y_valid = np.array(to_categorical(valid.label), dtype=int)
y_test = np.array(to_categorical(test.label), dtype=int)

max_epoch = 7

In [ ]:
from tensorflow.keras.callbacks import Callback
import matplotlib.pyplot as plt

class VisualiseAttentionMap(Callback):

        def on_epoch_end(self, epoch, logs=None):
            concatenate_layer_out = Model(inputs=model.input, outputs=model.get_layer('concatenate').output)
            outputs = concatenate_layer_out.predict([test_sent1, test_sent2])

            plt.imshow(outputs[0][:MAX_SEQ_LENGTH], cmap='Blues')

            iteration_no = str(epoch).zfill(3)
            plt.axis('off')
            plt.title(f'Iteration {iteration_no} / {max_epoch} Sentence1')
            plt.savefig('attention1.png')
            plt.show()
            plt.close()
            plt.clf()

            plt.imshow(outputs[0][MAX_SEQ_LENGTH:], cmap='hot')

            iteration_no = str(epoch).zfill(3)
            plt.axis('off')
            plt.title(f'Iteration {iteration_no} / {max_epoch} Sentence2')
            plt.savefig('attention2.png')
            plt.show()
            plt.close()
            plt.clf()

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

checkpoint_path = '/content/drive/MyDrive/MCI/HW5/bilstm.ckpt'

cp_callback = [ModelCheckpoint(filepath=checkpoint_path, monitor='val_accuracy', mode = 'max', save_best_only=True), EarlyStopping(patience=10)]

In [ ]:
history = model.fit(
    [train_sent1, train_sent2],
    y_train,
    validation_data = ([valid_sent1, valid_sent2], y_valid),
    epochs=max_epoch,
    verbose=1,
    batch_size=128,
    callbacks=[VisualiseAttentionMap(), cp_callback])

In [ ]:
plt.style.use('ggplot')

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
predictions = model.predict([test_sent1, test_sent2])

pred_labels = [np.argmax(predictions[i]) for i in range(len(y_test))]

y_test = test.label.astype(int)

In [ ]:
from sklearn.metrics import *

print(classification_report(y_test, pred_labels, target_names=['Class 0', 'Class 1']))

In [ ]:
print("Test Accuracy:", accuracy_score(y_test, pred_labels)) 
print('Test Precision (micro):', precision_score(y_test, pred_labels, average='micro'))
print('Test Recall (micro):', recall_score(y_test, pred_labels, average='micro'))
print('Test F1 Score (micro):', f1_score(y_test, pred_labels, average='micro'))
print('Test Precision (macro):', precision_score(y_test, pred_labels, average='macro'))
print('Test Recall (macro):', recall_score(y_test, pred_labels, average='macro'))
print('Test F1 Score (macro):', f1_score(y_test, pred_labels, average='macro'))